In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time
plt.rcParams['font.family'] = 'Malgun Gothic'

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

def get_sales(dept_num, item_num, store_id): 
    df_train = pd.read_csv('./data/sales_train_df.csv', index_col=0) # 본인 파일 경로로 수정해야함
    tmp = df_train[df_train['cat_id'] == 'FOODS'] # df_train에서 카테고리가 식품인 것만 마스킹
    

    if len(str(item_num)) == 1: # item_num 글자수에 따라 자릿수 3자리 맞춰주기 (8 -> 008)
        tmp = df_train[df_train['item_id'] == f'FOODS_{dept_num}_00{item_num}']
    elif len(str(item_num)) == 2:
        tmp = df_train[df_train['item_id'] == f'FOODS_{dept_num}_0{item_num}']
    else:
        tmp = df_train[df_train['item_id'] == f'FOODS_{dept_num}_{item_num}']

    d_ = [col for col in df_train.columns if "d_" in col]
    store_sales = tmp.groupby('store_id')[d_].sum().T # 주별로 그룹
    store_sales = store_sales.iloc[:1908,:]
    return store_sales[store_id].values # 입력한 주의 판매량만 반환

def predict_item_id_sales(store_id, dept_num, item_id, weekday, event, Temperatures, pop, unemployment_rate):
    if (dept_num > 3) or (item_id < 1) or(dept_num == 1 and item_id > 219) or (dept_num == 2 and item_id > 399) or (dept_num == 3 and item_id > 827):
        raise Exception('범위 에러: FOODS_1은 1~ 219, FOODS_2은 1~ 399, FOODS_3은 1~ 827 으로 다시 입력해주세요.')
    
    X_train = pd.read_csv('./variable_train_df.csv', index_col=0) # 본인 파일 경로로 수정해야함
    state = X_train[X_train['state_id'] == store_id[:2]]
    X = state[['weekday','event','평균기온', '매장당평균인구수', '실업률']]
    y= [i for i in get_sales(dept_num, item_id, store_id)]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=13)

    lr1 = GradientBoostingRegressor()
    model = lr1.fit(X_train, y_train)
    y_pred = model.predict(pd.DataFrame(data={'weekday': [weekday, weekday], 'event':[event, event], '평균기온': [Temperatures, Temperatures], '매장당평균인구수': [pop, pop], '실업률': [unemployment_rate, unemployment_rate]}))
    return round(y_pred[0])


In [30]:
predict_item_id_sales('CA_3', 3, 90, 1, 1, 30, 120800, 10)

133

In [37]:
def print_result():
    print('===========================================')
    print('안녕하세요 판매량 예측 모델, 판담입니다.')
    print('특정 일의 판매량 예측을 위한 8가지 질문을 시작하겠습니다.')
    print('')

    print('1번) CA, TX, WI 중 원하시는 주를 입력해주세요.')
    state_id = input('주 입력 : ')
    
    if state_id == 'CA':
        print('1번) 1 ~ 4 중 원하시는 지점 번호를 입력해주세요.')
        store_num = input('지점 번호 입력 : ')
        
    else:
        print('1번) 1 ~ 3 중 원하시는 지점 번호를 입력해주세요.')
        store_num = input('지점 번호 입력 : ')
        
    store_id = state_id + '_' + store_num 
    print('')

    print('2번) 1 ~ 3 중 원하시는 식품 분류 번호를 입력해주세요.')
    dept_num = int(input('식품 분류 번호 입력 : '))
    print('')

    if dept_num==1:
        print('3번) 1 ~ 219 중 원하시는 식품 id를 입력해주세요.')
        item_id = int(input('식품 id 입력 : '))

    elif dept_num==2:
        print('3번) 1 ~ 399 중 원하시는 식품 id를 입력해주세요.')
        item_id = int(input('식품 id 입력 : '))

    elif dept_num==3:
        print('3번) 1 ~ 827 중 원하시는 식품 id를 입력해주세요.')
        item_id = int(input('식품 id 입력 : '))
    else : raise Exception('범위 에러: FOODS_1은 1~ 219, FOODS_2은 1~ 399, FOODS_3은 1~ 827 으로 다시 입력해주세요.')
    print('')

    print('4번) 평일->0, 주말->1 중 원하시는 날짜의 구분 숫자를 입력해주세요.')
    weekday = int(input('평일/주말 구분 숫자 입력 : '))
    print('')

    print('5번) 일반->0, 행사->1 중 원하시는 날짜의 행사유무 숫자를 입력해주세요.')
    event = int(input('행사 유무 숫자 입력 : '))
    print('')

    print('6번) 원하시는 날짜의 기온을 입력해주세요.')
    Temperatures = int(input('기온 입력(°C): '))
    print('')

    print('7번) 원하시는 날짜의 매장 당 인구 수를 입력해주세요.')
    pop = int(input('매장 당 인구 입력(명): '))
    print('')

    print('8번) 원하시는 날짜의 실업률을 입력해주세요.')
    unemployment_rate = int(input('실업률 입력(%): '))
    print('')

    print('========= 결과 출력 중 잠시만 기다려주세요.===========')
    print('')
    result = predict_item_id_sales(store_id, dept_num, item_id, weekday, event, Temperatures, pop, unemployment_rate)
    print(f'입력하신 조건에 해당하는 일의 {store_id}지점 food_{dept_num}_{item_id} 판매량은 {result}입니다.')
    print('')
    print('이용해주셔서 감사합니다.')
    print('===========================================')


In [ ]:
print_result()

안녕하세요 판매량 예측 모델, 판담입니다.
특정 일의 판매량 예측을 위한 8가지 질문을 시작하겠습니다.

1번) CA, TX, WI 중 원하시는 주를 입력해주세요.
주 입력 : TX
1번) 1 ~ 3 중 원하시는 지점 번호를 입력해주세요.


In [28]:
print('===========================================')
print('안녕하세요 판매량 예측 모델, 판담입니다.')
print('특정 일의 판매량 예측을 위한 8가지 질문을 시작하겠습니다.')
print('')

print('1번) CA, TX, WI 중 원하시는 주를 입력해주세요.')
state_id = input('주 입력 : ')
print('')

print('2번) 1 ~ 3 중 원하시는 식품 분류 번호를 입력해주세요.')
dept_num = int(input('식품 분류 번호 입력 : '))
print('')

if dept_num==1:
    print('3번) 1 ~ 219 중 원하시는 식품 id를 입력해주세요.')
    item_id = int(input('식품 id 입력 : '))
    
elif dept_num==2:
    print('3번) 1 ~ 399 중 원하시는 식품 id를 입력해주세요.')
    item_id = int(input('식품 id 입력 : '))
    
elif dept_num==3:
    print('3번) 1 ~ 827 중 원하시는 식품 id를 입력해주세요.')
    item_id = int(input('식품 id 입력 : '))
else : raise Exception('범위 에러: FOODS_1은 1~ 219, FOODS_2은 1~ 399, FOODS_3은 1~ 827 으로 다시 입력해주세요.')
print('')

print('4번) 평일->0, 주말->1 중 원하시는 날짜의 구분 숫자를 입력해주세요.')
weekday = int(input('평일/주말 구분 숫자 입력 : '))
print('')

print('5번) 일반->0, 행사->1 중 원하시는 날짜의 행사유무 숫자를 입력해주세요.')
event = int(input('행사 유무 숫자 입력 : '))
print('')

print('6번) 원하시는 날짜의 기온을 입력해주세요.')
Temperatures = int(input('기온 입력(°C): '))
print('')

print('7번) 원하시는 날짜의 매장 당 인구 수를 입력해주세요.')
pop = int(input('매장 당 인구 입력(명): '))
print('')

print('8번) 원하시는 날짜의 실업률을 입력해주세요.')
unemployment_rate = int(input('실업률 입력(%): '))
print('')

print('========= 결과 출력 중 잠시만 기다려주세요.===========')
print('')
result = predict_item_id_sales(state_id, dept_num, item_id, weekday, event, Temperatures, pop, unemployment_rate)
print(f'입력하신 조건에 해당하는 일의 food_{dept_num}_{item_id} 판매량은 {result}입니다.')
print('')
print('이용해주셔서 감사합니다.')
print('===========================================')


안녕하세요 판매량 예측 모델, 판담입니다.
특정 일의 판매량 예측을 위한 8가지 질문을 시작하겠습니다.

1번) CA, TX, WI 중 원하시는 주를 입력해주세요.
주 입력 : CA

2번) 1 ~ 3 중 원하시는 식품 분류 번호를 입력해주세요.
식품 분류 번호 입력 : 3

3번) 1 ~ 827 중 원하시는 식품 id를 입력해주세요.
식품 id 입력 : 90

4번) 평일->0, 주말->1 중 원하시는 날짜의 구분 숫자를 입력해주세요.
평일/주말 구분 숫자 입력 : 1

5번) 일반->0, 행사->1 중 원하시는 날짜의 행사유무 숫자를 입력해주세요.
행사 유무 숫자 입력 : 1

6번) 원하시는 날짜의 기온을 입력해주세요.
기온 입력(°C): 30

7번) 원하시는 날짜의 매장 당 인구 수를 입력해주세요.
매장 당 인구 입력(명): 120800

8번) 원하시는 날짜의 실업률을 입력해주세요.
실업률 입력(%): 10

========= 결과 출력 중 잠시만 기다려주세요.===========

입력하신 조건에 해당하는 일의 food_3_90 판매량은 409입니다.

이용해주셔서 감사합니다.
